In [ ]:
from sklearn.cluster import KMeans
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, GaussianNB
%matplotlib inline

In [ ]:
#nlp
import nltk
nltk.download('punkt')
import nltk
# Download NLTK stopwords data
nltk.download('stopwords')
import string  # Import the string module
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from sklearn import datasets
wine = datasets.load_wine()

In [10]:
#preprocessing
def spamCheck(df):
  df['Message'] = df['Message'].str.lower()
  df['Message'] = df['Message'].str.replace('[{}]'.format(string.punctuation), '')
  stop_words = set(stopwords.words('english'))
  df['Message'] = df['Message'].apply(lambda x: ' '.join([word for word in word_tokenize(x) if word not in stop_words]))
  stemmer = PorterStemmer()
  df['Message'] = df['Message'].apply(lambda x: ' '.join([stemmer.stem(word) for word in word_tokenize(x)]))
  df['Category'] = df['Category'].apply(lambda x : 0 if x=='ham' else 1)

In [9]:
df = pd.read_csv('spam.csv')
spamCheck(df)
X_list=df['Message'].tolist()
Y_list=df['Category'].tolist()
finalFeatures = []
frequentData=[]
for sentence in X_list:
  words = sentence.split()
  hashTable = {}
  for word in words:
    if word not in hashTable:
      if word not in finalFeatures:
        finalFeatures.append(word)
      hashTable[word]=1
    else:
      hashTable[word] = hashTable[word]+1
  frequentData.append(hashTable)
df=pd.DataFrame()
for data in frequentData:
  row=[]
  col=[]
  for key,value in data.items():
    row.append(value)
    col.append(key)
  tempDataFrame = pd.DataFrame([row],columns = col)
  df=df.append(tempDataFrame)
for feature in finalFeatures:
  df[feature].fillna(0, inplace = True)

_PB0 = 0.0
_PB1 = 0.0
for y in Y_list:
  if(y==0):
    _PB0 += 1
  else:
    _PB1 +=1

_PB0 = _PB0 / (len(Y_list))
_PB1 = _PB1 / (len(Y_list))
df['Spam']=Y_list
P0 = 1
P1 = 1
X = df[df['Spam']==0]
Y = df[df['Spam']==1]
# Y.head()
res1_lis = []
# print(finalFeatures)
for feature in finalFeatures:
  res1_lis.append(X[feature].sum())
res1 = pd.DataFrame([res1_lis],columns = finalFeatures)
res2_lis = []
for feature in finalFeatures:
  res2_lis.append(Y[feature].sum())
res1 = res1.append(pd.DataFrame([res2_lis],columns=finalFeatures))
res1 = res1.applymap(lambda y : y+1)
row_sum = res1.sum(axis=1)
res1['sum']=row_sum
res1.set_index(pd.RangeIndex(start=1, stop=len(res1) + 1, name='Index'), inplace=True)
res1.to_csv('output.csv',index=False)
res1.head()

Streaming output truncated to the last 5000 lines.
<ipython-input-9-2b99a3e3faa2>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(tempDataFrame)
<ipython-input-9-2b99a3e3faa2>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(tempDataFrame)
<ipython-input-9-2b99a3e3faa2>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(tempDataFrame)
<ipython-input-9-2b99a3e3faa2>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(tempDataFrame)
<ipython-input-9-2b99a3e3faa2>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conca

,go,jurong,point,crazi,avail,bugi,n,great,world,la,...,dental,nmde,dump,heap,salesman,£750,087187272008,now1,piti,soani
Index,,,,,,,,,,,,,,,,,,,,,
1,418.0,2.0,18.0,11.0,14.0,8.0,138.0,102.0,35.0,8.0,...,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0
2,36.0,1.0,17.0,6.0,4.0,1.0,10.0,12.0,3.0,1.0,...,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0


In [ ]:
test_df =  pd.read_csv('spam.csv')
spamCheck(test_df)
test_message = test_df['Message'].tolist()
test_category = test_df['Category'].tolist()
count=0
i=0
for message in test_message:
  if(testing(message) == test_category[i]):
    count+=1
  i+=1
print((count/len(test_df))*100)

In [19]:
def testing(test):
  p1 = 1
  for word in test.split():
    if word in finalFeatures:
      t1 = res1.at[1,word]
      t2 = res1.at[1,'sum']
      p1 = p1 * t1 / t2
  p1 = p1 * len( X )/ len(Y_list)
  p2 = 1
  for word in test.split():
    if word in finalFeatures:
      t1 = res1.at[2,word]
      t2 = res1.at[2,'sum']
      p2 = p2 * t1 / t2
  p2 = p2 * len( Y )/ len(Y_list)
  if(p1>p2):
    return 0
  else:
    return 1